In [2]:
# https://pypi.org/project/mysql-connector-python/
# !pip install mysql-connector-python

In [8]:
import mysql.connector as connection

In [14]:
try:
    mydb = connection.connect(host='localhost',
                             user='root',
                             password='root')
    print(f"Connected to : {mydb}")
    
    cursor = mydb.cursor()
    query = "show databases;"
    cursor.execute(query)
    print(f"{cursor.fetchall()}")
except Exception as e:
    mydb.close()
    print(f"Exception : {e}")
    

Connected to : <mysql.connector.connection_cext.CMySQLConnection object at 0x0000027AB4AAC730>
[('information_schema',), ('mysql',), ('performance_schema',), ('sakila',), ('sys',), ('world',)]


In [21]:
# CREATE DATABASE
query = 'CREATE database project_practice;'
cursor.execute(query)
print(f"CREATE DATABASE : {cursor.fetchall()}")

query = "show databases;"
cursor.execute(query)
print(f"SHOW DATABASE :")
for db_name in cursor.fetchall():
    print(f"{db_name[0]}")

# TO DROP/DELETE DATABASE
# query = 'drop database project_practice;'

CREATE DATABASE : []
SHOW DATABASE :
information_schema
mysql
performance_schema
project_practice
sakila
sys
world


In [27]:
# CREATE TABLE
mydb = connection.connect(host='localhost',
                          user='root',
                          password='root', 
                          database='project_practice')
cursor = mydb.cursor()
print(f"Connected to : {mydb} , {cursor}")

query = """create table if not exists student_details (stud_id int(10) auto_increment primary key, first_name varchar(20),
last_name varchar(20), registration_date date, class varchar(20), section varchar(20));"""
# query = """use project_practice;
# create table student_details (stud_id int(10) auto_increment primary key, first_name varchar(20),
# last_name varchar(20), registration_date date, class varchar(20), section varchar(20));"""
cursor.execute(query)
print(f"SHOW TABLE : {cursor.fetchall()}")

Connected to : <mysql.connector.connection_cext.CMySQLConnection object at 0x0000027AB4A67DF0> , CMySQLCursor: (Nothing executed yet)
SHOW TABLE : []


In [34]:
# INSERT DATA
query = "INSERT INTO student_details (first_name, last_name, registration_date, class, section) VALUES (%s, %s, %s, %s, %s);"
values = ("John", "Smith", "2000-06-16", "1", "A")

cursor.execute(query, values)

mydb.commit()

print(cursor.rowcount, "record inserted.")

1 record inserted.


In [35]:
query = "INSERT INTO student_details (first_name, last_name, registration_date, class, section) VALUES (%s, %s, %s, %s, %s);"
values = [
    ("Sam", "Smith", "2002-06-11", "2", "B"),
    ("Lily", "Prince", "2003-07-17", "5", "C"),
    ("Anna", "Moly", "2001-05-26", "2", "A"),
    ("Will", "Turnor", "2003-06-11", "3", "B"),
    ("Jacky", "Chan", "2002-06-20", "4", "A")
]
# To insert multiple rows into a table, use the executemany() method.
cursor.executemany(query, values)

mydb.commit()

print(cursor.rowcount, "record inserted.")

5 record inserted.


In [37]:
query = "select * from student_details;"
cursor.execute(query)
print(f"TABLE student_details :")
for db_name in cursor.fetchall():
    print(f"{db_name}")


TABLE student_details :
(1, 'John', 'Smith', datetime.date(2000, 6, 11), '2', 'A')
(2, 'John', 'Smith', datetime.date(2000, 6, 11), '2', 'A')
(3, 'John', 'Smith', datetime.date(2000, 6, 16), '1', 'A')
(4, 'Sam', 'Smith', datetime.date(2002, 6, 11), '2', 'B')
(5, 'Lily', 'Prince', datetime.date(2003, 7, 17), '5', 'C')
(6, 'Anna', 'Moly', datetime.date(2001, 5, 26), '2', 'A')
(7, 'Will', 'Turnor', datetime.date(2003, 6, 11), '3', 'B')
(8, 'Jacky', 'Chan', datetime.date(2002, 6, 20), '4', 'A')


# USE CASE : BANK MANAGEMENT SYSTEM

## FLOW CHART

1. register
2. login 

### MENU WILL BE DISPLAYED

1. create bank account
2. transaction
3. customer details
4. transaction details
5. delete account
6. quit

#### option 3 and 4 : data is fetch from the database

#### option 1, 2, 5, 6 : data insert in the database


In [12]:
import mysql.connector as connection
from datetime import datetime
import numpy as np
import uuid 

db_connection = connection.connect(host='localhost',
                          user='root',
                          password='root', 
                          database='bank_project')
db_cursor = db_connection.cursor()
print(f"Connected to {db_connection} : {db_connection.is_connected()} ")

Connected to <mysql.connector.connection_cext.CMySQLConnection object at 0x000001D439C830A0> : True 


In [20]:
def create_tables(db_connection, db_cursor):
    try:
#         CREATE USER TABLE
        query = """create table if not exists user_table(
        user_name varchar(25) primary key,
        user_password varchar(25) not null
        );"""
        db_cursor.execute(query)
        db_connection.commit()
#         print(f"Created user table successfully!")
        
#         CREATE CUSTOMER DETAILS TABLE 
        query = """create table if not exists customer_details_table(
        account_no varchar(50) primary key,
        account_name varchar(25) not null,
        phone_no bigint(25) check(phone_no > 9),
        address varchar(25),
        account_balance float not null
        );"""
        db_cursor.execute(query)
        db_connection.commit()
#         print(f"Created customer details table successfully!")
        
#         CREATE TRANSACTIONS TABLE
        query = """create table if not exists transactions_table(
        account_no varchar(50),
        transaction_date timestamp,
        withdrawal_amount float,
        deposit_amount float,
        balance_amount float
        );"""
        db_cursor.execute(query)
        db_connection.commit()
#         print(f"Created transactions table successfully!")
        
    except Exception as e:
        print(f"[create_tables] Exception : {e}")
        exit(1)
        '''
        Python has only two standard codes, i.e., a one and a zero. 
        The exit code of 0 means that the process has been executed and exited successfully. 
        This means no error was encountered. On the other hand, an error code of 1 indicates 
        that the process exited with a failure.
        '''

def register_new_user():
    print(f"PLEASE FILL THE REGISTATION DETAILS : ")
    user_name = input("USER NAME : ")
    user_password = input("PASSWORD : ")
    phone_no = int(input("PHONE NUMBER : "))
    address = input("ADDRESS : ")
    credite_amt = float(input("CREATE AMOUNT : ") or "0")
    account_no = str(uuid.uuid4()) # GENERATE ACCOUNT ID
    transaction_date = str(datetime.now()).split('.')[0]
    
#     CHECK IF USER NAME IS AVAILABLE OR NOT
    query = f"""select * from user_table where user_name = '{user_name}';"""
    db_cursor.execute(query)
    
#     INSERT USER DATA TO USER TABLE
    if len(db_cursor.fetchall()) == 0:
        query = f"""insert into user_table (user_name, user_password) 
        values ('{user_name}', '{user_password}');"""
        db_cursor.execute(query)
        db_connection.commit()
        
#         ADD CUSTOMER DETAILS IN CUSTOMER DETAIL TABLE 
        query = f"""insert into customer_details_table (account_no, account_name,
        phone_no, address, account_balance) values ('{account_no}', '{user_name}', 
        {phone_no}, '{address}', {credite_amt});"""
        db_cursor.execute(query)
        db_connection.commit()
        print(f"ADD CUSTOMER DETAILS IN CUSTOMER DETAIL TABLE : {db_cursor}")
        
#         CREATE 1ST TRANSACTION
        if credite_amt > 0:
#             GET BALANCE
#             query = """select balance_amount 
#             from transactions_table
#             order by transaction_date desc
#             limit 1;"""
#             db_cursor.execute(query)
#             if len(db_cursor.fetchall()) > 0:
#                 balance_amount = db_cursor.fetchall()[0] + credite_amt
#             else:
            balance_amount = credite_amt

#         INSERT TRANSACTION TO TRANSACTION TABLE
        query = f"""insert into transactions_table (account_no, 
        transaction_date, withdrawal_amount, deposit_amount, balance_amount) 
        values ('{account_no}', '{transaction_date}', {0}, {credite_amt}, {balance_amount});"""
        db_cursor.execute(query)
        db_connection.commit()
        
        print(f"INSERT TRANSACTION TO TRANSACTION TABLE : {db_cursor}")
            
#             GO TO HOME PAGE ---X---X---

    else:
        print(f"USER NAME IS ALREADY TAKEN. PLEASE TRY AGAIN...")
        
def login_user():
    print(f"PLEASE ENTER USERNAME AND PASSWORD : ")
    user_name = input("USER NAME : ")
    user_password = input("PASSWORD : ")
    
    query = f"""select user_name from user_table 
    where user_name = '{user_name}' 
    and user_password = '{user_password}';"""
    db_cursor.execute(query)
    
    if len(db_cursor.fetchall()) > 0:
        query = f"""insert into user_table (user_name, user_password) 
        values ('{user_name}', '{user_password}');"""
        db_cursor.execute(query)
        db_connection.commit()
        
#         GO TO HOME PAGE ---X---X---

    else:
        print(f"SORRY, WRONG USERNAME AND PASSWORD. PLEASE TRY AGAIN...")
        
def login_register(choice):
#     CHECK IF REGISTER
    if choice == 1:
        register_new_user()
#     LOGIN USER
    elif choice == 2:
        login_user()
    

In [21]:
def main():
    create_tables(db_connection, db_cursor)
    print(f"----- WELLCOME TO MYSQL BANK -----")
    while True:
        print(f"1. REGISTER AS NEW USER. \n2. LOGIN AS USER. ")
        try:
            login_register_choice = int(input("ENTER YOUR CHOICE PLEASE : "))
            if login_register_choice == 3:
                break
            login_register(login_register_choice)
        except Exception as e:
            print(f"[main] Exception : {e}")
            print(f"PLEASE TRY AGAIN...")

main()

----- WELLCOME TO MYSQL BANK -----
1. REGISTER AS NEW USER. 
2. LOGIN AS USER. 
ENTER YOUR CHOICE PLEASE : 1
PLEASE FILL THE REGISTATION DETAILS : 
USER NAME : shino
PASSWORD : yagamoto
PHONE NUMBER : 6547893210
ADDRESS : vasai
CREATE AMOUNT : 23
ADD CUSTOMER DETAILS IN CUSTOMER DETAIL TABLE : CMySQLCursor: insert into customer_details_table (acco..
INSERT TRANSACTION TO TRANSACTION TABLE : CMySQLCursor: insert into transactions_table (account_..
1. REGISTER AS NEW USER. 
2. LOGIN AS USER. 
ENTER YOUR CHOICE PLEASE : 3


In [109]:
len(str(uuid.uuid4()))

# https://pypi.org/project/shortuuid/

36

In [108]:
# db_connection = connection.connect(host='localhost',
#                           user='root',
#                           password='root', 
#                           database='bank_project')
# db_cursor = db_connection.cursor()
# print(f"Connected to {db_connection} : {db_connection.is_connected()} ")

# query = f"""drop table if exists customer_details_table;
# drop table if exists transactions_table;
# drop table if exists user_table;
# """
# db_cursor.execute(query)
# db_connection.commit()


Connected to <mysql.connector.connection_cext.CMySQLConnection object at 0x0000027AB6C2BBE0> : True 


MySQLInterfaceError: Commands out of sync; you can't run this command now

In [2]:
# !pip install shortuuid


[notice] A new release of pip is available: 23.1.1 -> 23.2.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [8]:
import shortuuid 
su = shortuuid.ShortUUID(alphabet="0123456789")
su.uuid()
# su.get_alphabet()

'303127057992520530008490291122369036612'

In [ ]:
# https://stackoverflow.com/questions/31309594/check-if-query-run-successfully-through-python#:~:text=echo%20%24%3F,1%20if%20its%20not%20successful.
# https://www.datacamp.com/tutorial/tutorial-how-to-execute-sql-queries-in-r-and-python


# ========================================================

        if credite_amt > 0:
#             GET BALANCE
#             query = """select balance_amount 
#             from transactions_table
#             order by transaction_date desc
#             limit 1;"""
#             db_cursor.execute(query)
#             if len(db_cursor.fetchall()) > 0:
#                 balance_amount = db_cursor.fetchall()[0] + credite_amt
#             else:
            balance_amount = credite_amt
